# Psi method simulation 

## Explination

The Psi method is an adaptive method to estimate the threshold and slope of a psychometric function using Bayesian estimation. Most of the concepts in this notebook is taken from Kontesevich and Tyler 1997 from Vision Research. 

The threshold of the psychometric function, denoted by $\alpha$, is the inflection point of the function, or the point at wich the response is 50/50 (in this specific case). It represents the bias in the estimates. The slope, denoted by $\beta$, reflects the uncertainty in the estimate.

The Psi method is an adaptive method in that the next stimulus is selected based on the previous response(s). This is in constrast to the method of constant stimuli which provides the same stimuli a pre-set number of times. Adaptive methods need to address three main considerations:
1. Estimate of psychometric parameters
2. The termination rule
3. Placement of the next trial

1. Estimate the psychometric parameters ($\alpha$ and $\beta$) 

    The most efficient way to obtain these estimates is to use Bayesian estimation. First, we define the 2-d prior by making an initial guess for each of the parameters:
    $$ p(\alpha, \beta)$$
    Then, after the response, $r$, we calculate the probability of all prior responses, given each set of parameter values:
    $$ p(r | \alpha, \beta)$$
    Multiplying these distributions together yields the posterior distribution of all possible parameter values:
    $$ p(\alpha, \beta | r)$$

    The mean of this distribution is the best estimate of the parameters because it minimizes the variance of the threshold and is more stable than using the maximum a posteri. 

    The posterior then becomes the prior for the next trial, and the process repeats until the test is terminated. 

2. Termination rule

    There are several different options, for simplicity, we plan on stopping after a certain number of trials

3. Placement of the next trial

    This is where the psi algorithm really does its work. To estimate threshold the most optimal location for the next stimulus is at the current threshold estimate, for the slope, it is slightly above and below the stimulus (in general). But, what specific stimulus should be chosen? Before each trial, the Psi algorithm simulates over every possible stimulus location ($x$) for the next trial, calculating the posterior distribution for each stimulus location for both possible responses. Then, it calculates the Information Entropy ($H$) for each stimulus location ($H(x)$). The next stimulus is the one that minimizes the entropy" $\underset{x}{argmin}(H(x))$

## Code

First, we need to define a look-up table to speed up the method. This look-up table contains conditional probabilities of both possible responses given the parameter values and the stimulus location:

$$p(r| \alpha,\beta,x)$$

This probability is defined by:

$$p(r=left | \alpha,\beta,x) = \Psi_{\alpha,\beta}(x)$$ 
and
$$p(r=right | \alpha,\beta,x) = 1-\Psi_{\alpha,\beta}(x)$$ 

In [11]:
#Import python notebooks
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm

In [16]:
def Psi_lookT(X,a_range,b_range):
    g = 0.02
    L = 0.02
    
    p_left_look = np.empty([len(b_range),len(a_range),(len(X))])
    p_right_look = np.empty([len(b_range),len(a_range),(len(X))])
    for x in X:
        for a in a_range:
            for b in b_range:
                psi = g + (1 - g - L) * norm.cdf(x,a,b)
                p_left_look[b,a,x] = psi
                p_right_look[b,a,x] = 1-psi

    return p_left_look, p_right_look

In [17]:
#Define the stimulus range
X = np.arange(-50,51)
a_range = np.arange(-50,51)
b_range = np.arange(0,101)

p_left_look, p_right_look = Psi_lookT(X,a_range,b_range)

C:\Users\Jonathan\Anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:1735: RuntimeWarning: invalid value encountered in long_scalars
  x = np.asarray((x - loc)/scale, dtype=dtyp)
C:\Users\Jonathan\Anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:879: RuntimeWarning: invalid value encountered in greater
  return (self.a < x) & (x < self.b)
C:\Users\Jonathan\Anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:879: RuntimeWarning: invalid value encountered in less
  return (self.a < x) & (x < self.b)
C:\Users\Jonathan\Anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:1738: RuntimeWarning: invalid value encountered in greater_equal
  cond2 = (x >= self.b) & cond0
C:\Users\Jonathan\Anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:1735: RuntimeWarning: divide by zero encountered in long_scalars
  x = np.asarray((x - loc)/scale, dtype=dtyp)
